In [1]:
import os
from os.path import isfile, join, abspath, basename
from optparse import OptionParser
import sys
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
import json

# functions to read and dump csv to sql server
def postgresql_engine_generator_mvesc(pass_file="/mnt/data/mvesc/pgpass"):
    """ Create postgres engine from credential-file
    Note: you can only run it on the mvesc-AWS-server
    :param str pass_file: file with the credential information
    :return sqlalchemy.engine object engine: object created create_engine() in sqlalchemy
    :rtype sqlalchemy.engine
    """
    with open(pass_file, 'r') as f:
        passinfo = f.read()
    passinfo = passinfo.strip().split(':')
    host_address = passinfo[0]
    port = passinfo[1]
    user_name = passinfo[2]
    name_of_database = passinfo[3]
    user_password = passinfo[4]
    sql_eng_str = "postgresql://"+user_name+":"+user_password+"@"+host_address+'/'+name_of_database
    engine = create_engine(sql_eng_str)
    return engine

In [2]:
engine = postgresql_engine_generator_mvesc()

